This is notebook file is the Solution for the Exercise from the [exercise series 4](https://github.com/righetti/optlearningcontrol/tree/master/Spring2020/series4) for the course Optimal and Learning Control (ME-GY 7973), as solved by Tarun Thathvik Paladugu. All the content from the file [Series 4.ipynb](https://github.com/righetti/optlearningcontrol/blob/master/Spring2020/series4/Series%204.ipynb) was used as provided by professor [Ludovic Righetti](https://engineering.nyu.edu/faculty/ludovic-righetti).

In [1]:
# a few packages we need to import

%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython 

import itertools

# Exercise 1 - REINFORCE

The goal of this exercise is to implement the simplest policy gradient algorith, REINFORCE, and then augment it with a baseline coming from an approximation of the value function. We will use this algorithm on the inverted pendulum model. The class below defines the pendulum model and some helper function to simulate it.

In [2]:
class Pendulum:
    """
    This class describes an inverted pendulum and provides some helper functions
    """
    
    def __init__(self):
        """
        constructor of the class
        """
        #gravity constant
        self.g=9.81
        
        # number of dimensions (angle and angular velocity)
        self.num_states = 2
        
        # the available control inputs
        self.controls = np.array([-5., 0., 5.])
        self.nb_controls = len(self.controls)

        # the maximum velocity
        self.vmax = 6.
        # the range of allowable states
        self.state_range = np.array([[0, 2*np.pi],[-self.vmax, self.vmax]])

        #simulation step
        self.delta_t = 0.1
        # internal integration step
        self._internaldt = 0.01
        self._integration_ratio = int(round(self.delta_t / self._internaldt))
        
            
    def step(self,x,u):
        """
        This function integrates the pendulum for one step of self.delta_t seconds
        
        Inputs:
        x: state of the pendulum (x,v) as a 2D numpy array
        u: control as a scalar
        
        Output:
        the state of the pendulum as a 2D numpy array at the end of the integration
        """
        x_next = x[0]
        v_next = x[1]
        for i in range(self._integration_ratio):
            x_next = (x_next + self._internaldt * v_next)%(2*np.pi)
            v_next = np.clip(v_next + self._internaldt * (u-self.g*np.sin(x_next)), -self.vmax, self.vmax)
        return np.array([x_next,v_next])
    
    def simulate(self, x0, policy, T):
        """
        This function simulates the pendulum for T seconds from initial state x0 using a policy
        (policy is called as policy(x) and returns one control)
        
        Inputs:
        x0: the initial conditions of the pendulum as a 2D array (angle and velocity)
        T: the time to integrate for
        
        Output:
        x (2D array) and u (1D array) containing the time evolution of states and control
        """
        horizon_length = int(T/self.delta_t)
        x=np.empty([2, horizon_length+1])
        x[:,0] = x0
        u=np.empty([horizon_length])
        for i in range(horizon_length):
            u[i] = policy(x[:,i])
            x[:,i+1] = self.step(x[:,i], u[i])
        return x, u

In [3]:
def animate_pendulum(x, dt):
    """
    This function makes an animation showing the behavior of the pendulum
    takes as input the result of a simulation - dt is the sampling time (0.1s normally)
    """
    
    # here we check if we need to down-sample the data for display
    #downsampling (we want 100ms DT or higher)
    min_dt = 0.1
    if(dt < min_dt):
        steps = int(min_dt/dt)
        use_dt = int(min_dt * 1000)
    else:
        steps = 1
        use_dt = int(dt * 1000)
    plotx = x[:,::steps]
    
    fig = matplotlib.figure.Figure(figsize=[6,6])
    matplotlib.backends.backend_agg.FigureCanvasAgg(fig)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=[-1.3,1.3], ylim=[-1.3,1.3])
    ax.grid()
    
    list_of_lines = []
    
    #create the cart pole
    line, = ax.plot([], [], 'k', lw=2)
    list_of_lines.append(line)
    line, = ax.plot([], [], 'o', lw=2)
    list_of_lines.append(line)
    
    cart_height = 0.25
    
    def animate(i):
        for l in list_of_lines: #reset all lines
            l.set_data([],[])
        
        x_pend = np.sin(plotx[0,i])
        y_pend = -np.cos(plotx[0,i])
        
        list_of_lines[0].set_data([0., x_pend], [0., y_pend])
        list_of_lines[1].set_data([x_pend, x_pend], [y_pend, y_pend])
        
        return list_of_lines
    
    def init():
        return animate(0)


    ani = animation.FuncAnimation(fig, animate, np.arange(0, len(plotx[0,:])),
        interval=use_dt, blit=True, init_func=init)
    plt.close(fig)
    plt.close(ani._fig)
    IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))

This function can be used to plot the results of a simulation of the pendulum

In [4]:
def plot_results(pend, val, pol, policy, animate=True):
    """
    This function plots the results. It displays the value function, the policy for all states.
    Then it integrates the pendulum from state [0,0] and displays the states and control as a function of time
    Finally it shows an animation of the result
    """
    x0 = np.array([0.,0.])

    x, u = pend.simulate(x0, policy, 10)

    plt.figure(figsize=[6,6])
    plt.imshow(val, extent=[0., 2*np.pi, -6, 6], aspect='auto')
    plt.xlabel('Pendulum Angle')
    plt.ylabel('Velocity')
    plt.title('Value Function')

    plt.figure(figsize=[6,6])
    plt.imshow(pol, extent=[0., 2*np.pi, -6, 6], aspect='auto')
    plt.xlabel('Pendulum Angle')
    plt.ylabel('Velocity')
    plt.title('Policy')

    time = np.linspace(0.,20., len(x[0,:]))
    plt.figure()
    plt.subplot(3,1,1)
    plt.plot(time,x[0,:])
    plt.ylabel('angle')
    plt.subplot(3,1,2)
    plt.plot(time,x[1,:])
    plt.ylabel('velocity')
    plt.subplot(3,1,3)
    plt.plot(time[:-1],u)
    plt.ylabel('control')
    if animate:
        animate_pendulum(x, pend.delta_t)

## Reinforcement learning problem

We would like to get the pendulum upright starting from the down rest position. It has two states, its angle $\theta$ (where $\theta=0$ means that the pendulum is down) and its angular velocity $\omega$. The state of the system is $x = [\theta^T \omega^T]^T$. We also assume that we have only 3 possible control inputs $[-5, 0, 5]$.

We formulate this problem as a finite horizon problem where we want to minimize the cost
$$\min \sum_{i=0}^{N} \alpha^i g(\theta_i, \omega_i, u_i)$$
where the instantaneous cost is $$g(x,v,u) = (x-\pi)^2 + 0.01 v^2 + 0.00001 u^2$$
which gives a high cost for states far from $\pi$ (i.e. far from the inverted position) or states with non zero velocity or high controls

## Stochastic policies

We will use stochastic policies $\pi(u | x,\theta_\pi)$ that are parametrized by $\theta_\pi$ (i.e. the parameters to learn). We will use exponential softmax distributions $$\pi(u|x,\theta) = \frac{e^{h(x,u,\theta_\pi)}}{\sum_a e^{h(x,a,\theta)}}$$ where in the denominator we sum over the 3 possible controls.
The function $h(x,u,\theta_\pi)$ is a function which reflects the preference of a control for a given state. We will use a linear combination of nonlinear basis functions to define it as
$$h(x,u,\theta_\pi) = \theta_\pi^T \Psi(x,u)$$
where $$\Psi(x,u) = [\psi_0(x,u), \cdots, \psi_m(x,u)]^T$$ is a vector of $m$ nonlinear basis functions.
In our exercise, we use periodic basis functions. For a given order $p$, we define $m = 6 (p+1)^2$ basis functions as
$$\psi_{k,l,c,0}(\theta,\omega,u) = \frac{\mathrm{e}^{-\frac{(u-u_c)^2}{0.002}}}{\sqrt{2\pi 0.001}} \cos(k \theta + l \frac{\pi}{\omega_{max}}\omega)$$
and
$$\psi_{k,l,c,1}(\theta,\omega,u) = \frac{\mathrm{e}^{-\frac{(u-u_c)^2}{0.002}}}{\sqrt{2\pi 0.001}} \sin(k \theta + l \frac{\pi}{\omega_{max}}\omega)$$
where $0\leq k \leq p$ and $0 \leq l \leq p$ are integers, $u_c$ is the control indexes by $c$ (i.e. it is either -5, 0 or 5) and $\omega_{max}$ is the maximum velocity of the pendulum (in order to get one period over the full range of $\omega$). Note that $\theta$ already ranges from $0$ to $2\pi$.

The class below defines these stochastic policies as well as helper functions to sample from the policy, get the vector of basis functions and get the probability distribution $\pi(.|x,\theta_\pi)$

In [5]:
class StochasticPolicyPeriodicFeatures:
    """
    This class implements a stochastic policy with linear sum of nonlinear features
    the features are periodic functions multiplied by a radial basis function of u
    """
    def __init__(self, controls, order = 2):
        """
        class constructor - controls is the array of control inputs, order is the order of the periodic basis
        """
        self.controls =  controls.copy()
        self.num_controls = len(self.controls)
        self.exp_basis = np.zeros([self.num_controls])
        self.order = order
        
        # the vector of basis functions
        self.basis_vector = np.zeros([2*self.num_controls*(self.order+1)**2])
        
        # the linear parameters to learn
        self.theta = np.zeros_like(self.basis_vector)
        
    def basis(self, x, u):
        """
        Returns the vector of basis functions evaluated at x,u
        """
        dx = x[0]
        dy = x[1]/6. * np.pi
        count = 0
        for c in self.controls:
            du = 1/(np.sqrt(2*np.pi*0.001)) * np.exp(-(u-c)**2/0.002)
            for j,k in itertools.product(range(self.order+1), range(self.order+1)):
                self.basis_vector[count] = du * np.cos(j*dx + k*dy)
                self.basis_vector[count+1] = du * np.sin(j*dx + k*dy)
                count += 2
        return self.basis_vector

    def get_distribution(self, x):
        """ 
        Computes pi(u|x) for all u 
        returns an array of pi and an array of basis functions (row is the control index and column is the )
        """
        dist = np.zeros_like(self.controls)
        basis_fun = np.zeros([len(self.theta), len(self.controls)])
        for i,u in enumerate(self.controls):
            # this gives the basis function evaluated as (x,u)
            basis_fun[:,i] = self.basis(x,u)
            # dist gives exp(theta * basis_function)
            dist[i] = np.exp(self.theta.dot(basis_fun[:,i]))

        # we sum the exponentials
        sm = np.sum(dist)
        # dist is rescaled by the sum of exponentials (we now have a probability distribution)
        dist = dist/sm
        return dist, basis_fun
    
    def sample(self, x):
        """
        sample from the stochastic policy given x
        it returns the index of the control and its value
        """
        probs, basis = self.get_distribution(x)
        index = np.random.choice(len(self.controls), p=probs)
        return index, self.controls[index]

## Value function approximation

We also use a function approximator for the value function (used for the REINFORCE algorithm with baseline). We will write our approximation of the value function as
$$V(x,\theta_V) = \theta_V^T B(x)$$
where the vector of basis functions $B$ is also constituted of periodic functions of the form
$$b_{k,l,0}(\theta,\omega) = \cos(k \theta + l \frac{\pi}{\omega_{max}}\omega)$$ and
$$b_{k,l,1}(\theta,\omega) = \sin(k \theta + l \frac{\pi}{\omega_{max}}\omega)$$

The class below implements such a value function approximation

In [6]:
class ValueFunctionPeriodicFeatures:
    """
    This class implements a function approximator with linear sum of nonlinear features
    the features are periodic functions
    We use this to approximate the value function
    """
    
    def __init__(self, order = 2):
        """
        the class constructor - order is the order of the periodic basis
        """
        
        self.order = order
        self.basis_vector = np.zeros([2*(self.order+1)**2])

        # the parameters to learn
        self.theta = np.zeros_like(self.basis_vector)
        
    def basis(self, x):
        """
        Returns the vector of basis functions evaluated at x
        """
        dx = x[0]
        dy = x[1]/6. * np.pi
        count = 0
        for j,k in itertools.product(range(self.order+1), range(self.order+1)):
            self.basis_vector[count] = np.cos(j*dx + k*dy)
            self.basis_vector[count+1] = np.sin(j*dx + k*dy)
            count += 2
        return self.basis_vector
    
    def getValue(self, x):
        """
        returns the value at x and the basis functions evaluated at x
        """
        bs = self.basis(x)
        return bs.dot(self.theta), bs

## Questions

a) Compute $\nabla V(x, \theta_V)$ with respect to $\theta_V$ for a generic function $ V(x, \theta_V) = \theta_V^T B(x)$

b) Compute $\nabla \ln \pi(u_t | x_t, \theta_\pi)$ with respect to $\theta_\pi$ for a stochastic policy parametrized as $\pi(u|x,\theta) = \frac{e^{h(x,u,\theta_\pi)}}{\sum_a e^{h(x,a,\theta)}}$, with $h(x,u,\theta_\pi) = \theta_\pi^T \Psi(x,u)$

c) using the previous computations, the classes above (optional) and code skeleton below (also optional), implement the REINFORCE algorithm for the inverted pendulum. Use $\alpha = 0.99$, an episode length of 100 (i.e. 10 seconds) and order p=2 for the policy approximation. Hint: you may want to choose a learning rate between $10^{-8}$ and $10^{-11}$. How many episodes does it take to converge? Can the resulting policy (which is stochastic) balance the pendulum appropriately? (note you may want to test several times due to the stochasticity). Plot the learning progress (costs versus episodes), you may also want to use a moving average over the costs to remove noise.

d) implement the REINFORCE algorithm with baseline and answer the same questions as in c) (also use an order p=2 for the value function approximation). Additionally, plot the value function. Hint: you may want to choose a learning rate between $10^{-6}$ and $10^{-9}$ for updating the policy and a learning rate between $10^{-1}$ and $10^{-3}$ to update the value function.

e) Compare the convergence rate of both algorithms. Which one seem to be easier to use?

You can use the code skeleton below to answer the questions

#### Part a)
\begin{equation} 
\begin{split}
    \nabla V(x, \theta_V) =& \frac{\partial }{\partial \theta_V}V(x, \theta_V) \\
    =>\nabla V(x, \theta_V) =& \frac{\partial }{\partial \theta_V} \theta_V^T B(x)\\ 
    =>\nabla V(x, \theta_V) =&  B(x)
\end{split}
\end{equation}

In [7]:
def get_Value_Gradient(value, x):
    return value.basis(x)

#### Part b)
This is the mathematical derivation to find the gradient which will be used in the python function defined below:
\begin{equation} 
\begin{split}
\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{\partial }{\partial \theta_\pi}\ln\pi(u|x,\theta) \\ 
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{1}{\pi(u_t | x_t, \theta_\pi)}\frac{\partial }{\partial \theta_\pi}\pi(u|x,\theta)\\
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{1}{\pi(u_t | x_t, \theta_\pi)}\frac{\partial }{\partial \theta_\pi}\frac{e^{h(x,u,\theta_\pi)}}{\sum_a e^{h(x,a,\theta)}} \text{, where}\ h(x,u,\theta_\pi) = \theta_\pi^T \Psi(x,u)\\
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{1}{\pi(u_t | x_t, \theta_\pi)}\frac{\frac{\partial}{\partial \theta_\pi}h(x,u,\theta_\pi)e^{h(x,u,\theta_\pi)}\sum_a e^{h(x,a,\theta)}- e^{h(x,u,\theta_\pi)}\sum_a \frac{\partial}{\partial \theta_\pi}h(x,a,\theta_\pi)e^{h(x,a,\theta)}}{{\sum_a e^{h(x,a,\theta)}}^{2}}\\
\end{split}
\end{equation}
Since $h(x,u,\theta_\pi) = \theta_\pi^T \Psi(x,u)$, we get $\frac{\partial }{\partial \theta_\pi}h(x,u,\theta_\pi) = \Psi(x,u)$
\begin{equation} 
\begin{split}
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{1}{\pi(u_t | x_t, \theta_\pi)}\frac{\Psi(x,u)e^{h(x,u,\theta_\pi)}\sum_a e^{h(x,a,\theta)}- e^{h(x,u,\theta_\pi)}\sum_a \Psi(x,a)e^{h(x,a,\theta)}}{{\sum_a e^{h(x,a,\theta)}}^{2}}\\
\text{Substituting}\ \pi(u|x,\theta) = \frac{e^{h(x,u,\theta_\pi)}}{\sum_a e^{h(x,a,\theta)}}, \\
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{\sum_a e^{h(x,a,\theta)}}{e^{h(x,u,\theta_\pi)}}\frac{\Psi(x,u)e^{h(x,u,\theta_\pi)}\sum_a e^{h(x,a,\theta)}- e^{h(x,u,\theta_\pi)}\sum_a \Psi(x,a)e^{h(x,a,\theta)}}{{\sum_a e^{h(x,a,\theta)}}^{2}}\\
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \frac{\Psi(x,u)\sum_a e^{h(x,a,\theta)}- \sum_a \Psi(x,a)e^{h(x,a,\theta)}}{{\sum_a e^{h(x,a,\theta)}}}\\
=>\nabla \ln \pi(u_t | x_t, \theta_\pi) =& \Psi(x,u)- \frac{\sum_a \Psi(x,a)e^{h(x,a,\theta)}}{{\sum_a e^{h(x,a,\theta)}}}\\
\end{split}
\end{equation}

In [8]:
def get_Policy_Gradient(policy,x,u):
    e = np.zeros_like(policy.controls)
    psis = np.zeros([len(policy.theta), len(policy.controls)])
    #exp_times_psi = np.zeros([len(policy.theta), len(policy.controls)])
    exp_times_psi = 0
    index = 0
    for i,a in enumerate(policy.controls):
        psis[:,i] = policy.basis(x,a)
        e[i] = np.exp(policy.theta.dot(psis[:,i]))
        exp_times_psi += e[i]*(psis[:,i])
        if(a==u):
            index = i
    sm = np.sum(e)
    sm_psi = np.sum(exp_times_psi)
    grad = psis[:,index] - sm_psi/(sm)
    return grad

#### Part c)
Implementing REINFORCE

## REINFORCE

We want to implement the REINFORCE algorithm (where an iteration is shown below):
1. Generate an episode $x_0, u_0, x_1, u_1, \cdots, x_N, u_N$, where $u_i$ is sampled from the current policy $\pi(-|x,\theta_\pi)$
2. For each step $t$ of the episode
 * compute the return $G_t = \sum{k=t}^T \alpha^{k-t}g(x_k, u_k)$
 * do gradient descent on the policy parameters $\theta_\pi \leftarrow \theta_\pi - \gamma_\pi \alpha^t G_t \nabla \ln \pi(u_t | x_t, \theta_\pi)$
 
where $\gamma_\pi$ is a learning rate

In [9]:
class Reinforce:
    """
    An implementation of the reinforce algorithm (with or without baseline)
    """
    
    def __init__(self, model, cost, policy, discount_factor=0.99, 
                 episode_length=100, policy_learning_rate = 0.000001):
        """
        the class constructor
        """
        self.model = model
        self.cost = cost

        self.policy = policy
        
        self.discount_factor = discount_factor
        self.episode_length = episode_length
        
        self.policy_learning_rate = policy_learning_rate
            
    def iterate(self, num_iter=1):          
        """
        the main loop
        """
        learning_progress = []
        # here we allocate some useful vectors
        x_traj = np.zeros([self.episode_length+1, self.model.num_states])
        u_traj = np.zeros([self.episode_length, 1])
        u_index = np.zeros([self.episode_length], dtype=np.int)
        cost_traj = np.zeros([self.episode_length])
        
        for i in range(num_iter):
            # generate an episode - start from 0
            x_traj[0,:] = np.zeros([2])
            
            if (i+1)%50 == 0:
                print(i+1)

            # TO COMPLETE #
            # you can use the step function of self.model (i.e. the pendulum) to get the next state
            
            T = self.episode_length
            #x_traj , u_traj = self.model.simulate(x_traj[0], self.get_sample_from_policy,T)
            x_traj , u_traj = self.model.simulate(x_traj[0,:], self.get_sample_from_policy,T*self.model.delta_t)
            x_traj = np.transpose(x_traj)
            G = np.zeros([T])
            for t in range(T):
                for k in range(t,T):
                    g = self.cost(x_traj[k,:],u_traj[k])
                    G[t] += self.discount_factor**(k-t)*g
                gradient = get_Policy_Gradient(self.policy,x_traj[t,:],u_traj[t])
                change = self.policy_learning_rate*(self.discount_factor**t)*G[t]*gradient
                self.policy.theta = self.policy.theta - change#[:,0,0]
                # now compute the returns Gt and update the policy parameters self.policy.theta through gradient descent
            # TO COMPLETE #

            # here we store the return at t=0 to get the learning progress
            learning_progress.append(G[0])
                
        return learning_progress
            
    def get_sample_from_policy(self, x):
        index, control = self.policy.sample(x)
        return control
    
    def get_Policy(self):
        """
        This helper function generate a 50x50 grid (theta x omega) with a policy (for display)
        the policy is computed as the expected control from pi
        we also compute a value function (to be used for the baseline part) - for now it is 0
        """
        n_discrete = 50
        pol = np.zeros([n_discrete,n_discrete])
        val = np.zeros([n_discrete,n_discrete])
        x_range = np.linspace(self.model.state_range[0,0], self.model.state_range[0,1], n_discrete)
        v_range = np.linspace(self.model.state_range[1,0], self.model.state_range[1,1], n_discrete)
        
        for i, x in enumerate(x_range):
            for j,v in enumerate(v_range):
                dist, basis = self.policy.get_distribution(np.array([x,v]))
                pol[i,j] = dist.dot(self.model.controls)
                # this can be used later to get a value estimate
                # val[i,j] = self.value.getValue(np.array([x,v]))[0] 
        return pol, val

def cost(x,u):
    """
    the cost function for the inverted pendulum
    """
    return (x[0] - np.pi)**2 + 0.01*x[1]**2 + 0.0001*u

In [10]:
# here we create all the objects
pendulum = Pendulum()
policy = StochasticPolicyPeriodicFeatures(controls = pendulum.controls, order = 2)
# to be used for the baseline
# value = ValueFunctionPeriodicFeatures(order = 2)
reinforce = Reinforce(pendulum, cost, policy, episode_length=750, 
                      discount_factor=0.99, policy_learning_rate = 0.0000000001)
# here we learn doing 300 episodes
lear = reinforce.iterate(1500) #you will likely need more than 300 episodes!

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500


In [11]:
# we plot the learning progress - also including a moving average (over 20 data points)
plt.figure()
plt.plot(lear)
ret = np.cumsum(lear)
win_avg = 20
ret[win_avg:] = ret[win_avg:] - ret[:-win_avg]
plt.plot(ret[win_avg - 1:] / win_avg)

# we get the policy/value as arrays
pol, val = reinforce.get_Policy()
# we create a controller using the policy to be used in the simulation
control_policy = lambda x : reinforce.policy.sample(x)[1]
# we plot results including a 10s simulation of the pendulum using the policy
plot_results(pendulum, val, pol, control_policy, animate=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Your browser does not support the video tag.

It is observed that at a $10^{-10}$ policy learning rate, the system comes close to converging after 600 Iterations, with a 750 episode length. After multiple tries it can be said that this method cannot balance the pendulumn, it does show close to sucessful implementation multiple times, but never full balance.

#### Part d)

Now we Implement REINFORCE algorithm with Baseline

## REINFORCE with baseline
 
 The REINFORCE with baseline algorithm, in addition, maintains an estimate of the value function. One iteration is
 1. Generate an episode $x_0, u_0, x_1, u_1, \cdots, x_N, u_N$, where $u_i$ is sampled from the current policy $\pi(-|x,\theta_\pi)$
 2. For each step $t$ of the episode
 * compute the return $G_t = \sum_{k=t}^T \alpha^{k-t}g(x_k, u_k)$
 * $\delta = G_t - V(x_t, \theta_V)$
 * do gradient ascent on the value parameters $\theta_V \leftarrow \theta_V + \gamma_V \alpha^t \delta \nabla V(x_t, \theta_V)$
 * do gradient descent on the policy parameters $\theta_\pi \leftarrow \theta_\pi - \gamma_\pi \alpha^t G_t \nabla \ln \pi(u_t | x_t, \theta_\pi)$
 
where $\gamma_\pi$ and $\gamma_V$ are learning rates

In [12]:
class Reinforce_Baseline:
    """
    An implementation of the reinforce algorithm (with or without baseline)
    """
    
    def __init__(self, model, cost, policy, value, discount_factor=0.99, 
                 episode_length=100, policy_learning_rate = 0.000001, value_learning_rate = 0.01):
        """
        the class constructor
        """
        self.model = model
        self.cost = cost

        self.policy = policy
        self.value = value
        
        self.discount_factor = discount_factor
        self.episode_length = episode_length
        
        self.policy_learning_rate = policy_learning_rate
        self.value_learning_rate = value_learning_rate

            
    def iterate(self, num_iter=1):          
        """
        the main loop
        """
        learning_progress = []
        # here we allocate some useful vectors
        x_traj = np.zeros([self.episode_length+1, self.model.num_states])
        u_traj = np.zeros([self.episode_length, 1])
        u_index = np.zeros([self.episode_length], dtype=np.int)
        cost_traj = np.zeros([self.episode_length])
        
        for i in range(num_iter):
            # generate an episode - start from 0
            x_traj[0,:] = np.zeros([2])
            
            if (i+1)%50 == 0:
                print(i+1)

            # TO COMPLETE #
            # you can use the step function of self.model (i.e. the pendulum) to get the next state
            
            N = self.episode_length
            #x_traj , u_traj = self.model.simulate(x_traj[0], self.get_sample_from_policy,T)
            x_traj , u_traj = self.model.simulate(x_traj[0,:], self.get_sample_from_policy,N*self.model.delta_t)
            x_traj = np.transpose(x_traj)
            G = np.zeros([N])
            for t in range(N):
                for k in range(t,N):
                    g = self.cost(x_traj[k,:],u_traj[k])
                    G[t] += self.discount_factor**(k-t)*g
                policy_gradient = get_Policy_Gradient(self.policy,x_traj[t,:],u_traj[t])
                value_gradient = get_Value_Gradient(value,x_traj[t,:])
                delta = G[t] - value.getValue(x_traj[t,:])[0]
                policy_change = self.policy_learning_rate*(self.discount_factor**t)*delta*policy_gradient
                value_change = self.value_learning_rate*(self.discount_factor**t)*delta*value_gradient
                self.value.theta = self.value.theta + value_change
                self.policy.theta = self.policy.theta - policy_change
                # now compute the returns Gt and update the policy parameters self.policy.theta through gradient descent
            # TO COMPLETE #

            # here we store the return at t=0 to get the learning progress
            learning_progress.append(G[0])
                
        return learning_progress
            
    def get_sample_from_policy(self, x):
        index, control = self.policy.sample(x)
        return control
    
    def get_Policy(self):
        """
        This helper function generate a 50x50 grid (theta x omega) with a policy (for display)
        the policy is computed as the expected control from pi
        we also compute a value function (to be used for the baseline part) - for now it is 0
        """
        n_discrete = 50
        pol = np.zeros([n_discrete,n_discrete])
        val = np.zeros([n_discrete,n_discrete])
        x_range = np.linspace(self.model.state_range[0,0], self.model.state_range[0,1], n_discrete)
        v_range = np.linspace(self.model.state_range[1,0], self.model.state_range[1,1], n_discrete)
        
        for i, x in enumerate(x_range):
            for j,v in enumerate(v_range):
                dist, basis = self.policy.get_distribution(np.array([x,v]))
                pol[i,j] = dist.dot(self.model.controls)
                # this can be used later to get a value estimate
                val[i,j] = self.value.getValue(np.array([x,v]))[0] 
        return pol, val

def cost(x,u):
    """
    the cost function for the inverted pendulum
    """
    return (x[0] - np.pi)**2 + 0.01*x[1]**2 + 0.0001*u

In [ ]:
# here we create all the objects
pendulum = Pendulum()
policy = StochasticPolicyPeriodicFeatures(controls = pendulum.controls, order = 2)
# to be used for the baseline
value = ValueFunctionPeriodicFeatures(order = 2)
reinforce_baseline = Reinforce_Baseline(pendulum, cost, policy, value, episode_length=500, 
                      discount_factor=0.99, policy_learning_rate = 0.000001, value_learning_rate = 0.01)
lear_b = reinforce_baseline.iterate(10000) 

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750


In [ ]:
# we plot the learning progress - also including a moving average (over 20 data points)
plt.figure()
plt.plot(lear_b)
ret = np.cumsum(lear_b)
win_avg = 20
ret[win_avg:] = ret[win_avg:] - ret[:-win_avg]
plt.plot(ret[win_avg - 1:] / win_avg)

# we get the policy/value as arrays
pol, val = reinforce_baseline.get_Policy()
# we create a controller using the policy to be used in the simulation
control_policy = lambda x : reinforce_baseline.policy.sample(x)[1]
# we plot results including a 10s simulation of the pendulum using the policy
plot_results(pendulum, val, pol, control_policy, animate=True)

Observing the learning progress graph, we can see that the graph converges at around 800, and YES, the <u>pendulumn balances itself</u> appropriately, when iterated for 700 times. Through several tries it is found that this is possible when the Episode length is 500 or more. 

#### Part e)

The convergence of the REINFORCE algorithm with baseline is better and makes it more easy to use with only a 500 episode length and therefore faster computation, as opposed to the (Monte Carlo) REINFORCE algorithm which needs a 750 episode length to only see a decent behaviour, close to balance but not actual balance. Larger episodes lengths makes make the computation more complex, therefore REINFORCE algorithm with baseline is easier to use, and additionally it successfully balances, unlike the other. 